# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### Questão 1

In [5]:
#Verificando dados faltantes
df.isna().any()

Unnamed: 0               False
data_ref                 False
id_cliente               False
sexo                     False
posse_de_veiculo         False
posse_de_imovel          False
qtd_filhos               False
tipo_renda               False
educacao                 False
estado_civil             False
tipo_residencia          False
idade                    False
tempo_emprego             True
qt_pessoas_residencia    False
renda                    False
dtype: bool

In [65]:
df.shape

(12427, 15)

In [7]:
#Excluindo possíveis linha duplicadas
df = df.drop_duplicates()

In [64]:
df.dropna(inplace = True)

In [66]:
train, test = train_test_split(df, test_size = 0.25)

### Questão 2

In [67]:
modelo = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia'

In [68]:
md = smf.ols(modelo, data = test)

In [40]:
#0
reg = md.fit_regularized(method = 'elastic_net'
                         , refit = True
                         , L1_wt = 0.0
                         , alpha = 0)

reg.summary()

NotImplementedError: 

In [69]:
#0.001
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , L1_wt = 0
                       , alpha = 0.001)

reg.summary()

NotImplementedError: 

In [23]:
#0.005
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , alpha = 0.005)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     200.9
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          3.52e-274
Time:                        20:42:25   Log-Likelihood:                -3386.9
No. Observations:                3091   AIC:                             6792.
Df Residuals:                    3083   BIC:                             6846.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.2221      0.114     63.405      0.000       6.999       7.445
sexo[T.M]                                            0.7872      0.028     28.356      0.000       0.733       0.842
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                              0.0749      0.028      2.705      0.007       0.021       0.129
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                              0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                         -0.0141      0.056     -0.249      0.803      -0.125       0.097
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                           0.0276      0.049      0.559      0.576      -0.069       0.125
idade                                                0.0058      0.002      3.757      0.000       0.003       0.009
tempo_emprego                            

In [24]:
#0.01
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , alpha =  0.01)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     393.2
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          5.14e-274
Time:                        20:42:32   Log-Likelihood:                -3398.9
No. Observations:                3091   AIC:                             6808.
Df Residuals:                    3087   BIC:                             6838.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.4864      0.040    188.602      0.000       7.409       7.564
sexo[T.M]                                            0.7764      0.028     28.061      0.000       0.722       0.831
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                              0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                               0          0        nan        nan           0           0
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                                0          0        nan        nan           0           0
idade                                                     0          0        nan        nan           0           0
tempo_emprego                            

In [25]:
#0.05
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , alpha =  0.05)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     398.8
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          3.05e-277
Time:                        20:43:22   Log-Likelihood:                -3391.5
No. Observations:                3091   AIC:                             6793.
Df Residuals:                    3087   BIC:                             6823.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.2761      0.061    119.711      0.000       7.157       7.395
sexo[T.M]                                            0.7881      0.028     28.633      0.000       0.734       0.842
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                              0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                               0          0        nan        nan           0           0
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                                0          0        nan        nan           0           0
idade                                                0.0057      0.001      3.867      0.000       0.003       0.009
tempo_emprego                            

In [26]:
#0.1
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , alpha =  0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     398.8
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          3.05e-277
Time:                        20:45:44   Log-Likelihood:                -3391.5
No. Observations:                3091   AIC:                             6793.
Df Residuals:                    3087   BIC:                             6823.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.2761      0.061    119.711      0.000       7.157       7.395
sexo[T.M]                                            0.7881      0.028     28.633      0.000       0.734       0.842
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                              0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                               0          0        nan        nan           0           0
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                                0          0        nan        nan           0           0
idade                                                0.0057      0.001      3.867      0.000       0.003       0.009
tempo_emprego                            

### Questão 3

In [70]:
#0
reg = md.fit_regularized(method = 'elastic_net'
                         , refit = True
                         , L1_wt = 1.0
                         , alpha = 0)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.350
Method:                 Least Squares   F-statistic:                     70.71
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          8.87e-272
Time:                        21:24:04   Log-Likelihood:                -3402.0
No. Observations:                3107   AIC:                             6854.
Df Residuals:                    3083   BIC:                             7005.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.3822      0.325     19.656      0.000       5.746       7.019
sexo[T.M]                                            0.7949      0.030     26.873      0.000       0.737       0.853
posse_de_veiculo[T.True]                             0.0400      0.028      1.414      0.157      -0.015       0.096
posse_de_imovel[T.True]                              0.0907      0.028      3.216      0.001       0.035       0.146
C(tipo_renda)[T.Bolsista]                            0.1180      0.365      0.324      0.746      -0.597       0.833
C(tipo_renda)[T.Empresário]                          0.1746      0.030      5.756      0.000       0.115       0.234
C(tipo_renda)[T.Servidor público]                    0.0079      0.044      0.181      0.857      -0.078       0.094
C(educacao, Treatment(2))[T.Primário]                0.1443      0.161      0.899      0.369      -0.170       0.459
C(educacao, Treatment(2))[T.Pós graduação]          -0.0015      0.298     -0.005      0.996      -0.586       0.583
C(educacao, Treatment(2))[T.Superior completo]       0.0710      0.028      2.542      0.011       0.016       0.126
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0304      0.065     -0.465      0.642      -0.158       0.098
C(estado_civil)[T.Separado]                          0.3557      0.159      2.236      0.025       0.044       0.668
C(estado_civil)[T.Solteiro]                          0.3112      0.154      2.026      0.043       0.010       0.612
C(estado_civil)[T.União]                            -0.0686      0.050     -1.383      0.167      -0.166       0.029
C(estado_civil)[T.Viúvo]                             0.3571      0.169      2.114      0.035       0.026       0.688
C(tipo_residencia)[T.Casa]                          -0.0337      0.109     -0.309      0.758      -0.248       0.180
C(tipo_residencia)[T.Com os pais]                    0.0680      0.124      0.547      0.584      -0.176       0.312
C(tipo_residencia)[T.Comunitário]                   -0.3893      0.212     -1.837      0.066      -0.805       0.026
C(tipo_residencia)[T.Estúdio]                       -0.0873      0.217     -0.402      0.688      -0.513       0.338
C(tipo_residencia)[T.Governamental]                 -0.0991      0.131     -0.754      0.451      -0.357       0.158
qtd_filhos                                          -0.3018      0.149     -2.025      0.043      -0.594      -0.010
idade                                                0.0067      0.002      4.149      0.000       0.004       0.010
tempo_emprego                                        0.0607      0.002     28.748      0.000       0.057       0.065
qt_pessoas_residencia                    

In [71]:
#0.001
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , L1_wt = 1.0
                       , alpha = 0.001)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     94.31
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          3.49e-277
Time:                        21:24:10   Log-Likelihood:                -3402.9
No. Observations:                3107   AIC:                             6844.
Df Residuals:                    3089   BIC:                             6959.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.4344      0.310     20.746      0.000       5.826       7.043
sexo[T.M]                                            0.7937      0.029     26.910      0.000       0.736       0.852
posse_de_veiculo[T.True]                             0.0390      0.028      1.381      0.167      -0.016       0.094
posse_de_imovel[T.True]                              0.0918      0.028      3.262      0.001       0.037       0.147
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                          0.1713      0.030      5.795      0.000       0.113       0.229
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]       0.0704      0.028      2.549      0.011       0.016       0.125
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0307      0.065     -0.471      0.637      -0.158       0.097
C(estado_civil)[T.Separado]                          0.3549      0.159      2.233      0.026       0.043       0.666
C(estado_civil)[T.Solteiro]                          0.3088      0.153      2.013      0.044       0.008       0.610
C(estado_civil)[T.União]                            -0.0683      0.050     -1.379      0.168      -0.165       0.029
C(estado_civil)[T.Viúvo]                             0.3598      0.169      2.133      0.033       0.029       0.691
C(tipo_residencia)[T.Casa]                          -0.0768      0.055     -1.400      0.162      -0.184       0.031
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                   -0.4333      0.190     -2.282      0.023      -0.806      -0.061
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                 -0.1430      0.091     -1.566      0.117      -0.322       0.036
qtd_filhos                                          -0.3025      0.149     -2.031      0.042      -0.595      -0.010
idade                                                0.0065      0.002      4.076      0.000       0.003       0.010
tempo_emprego                                        0.0607      0.002     29.064      0.000       0.057       0.065
qt_pessoas_residencia                    

In [74]:
#0.005
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , L1_wt = 1.0 
                       , alpha = 0.005)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     228.0
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          4.81e-274
Time:                        21:26:24   Log-Likelihood:                -3438.1
No. Observations:                3107   AIC:                             6892.
Df Residuals:                    3100   BIC:                             6940.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.1697      0.114     62.665      0.000       6.945       7.394
sexo[T.M]                                            0.7935      0.028     28.147      0.000       0.738       0.849
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                          0.0077      0.057      0.135      0.893      -0.105       0.120
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                           0.0149      0.049      0.302      0.762      -0.082       0.111
idade                                                0.0063      0.002      4.038      0.000       0.003       0.009
tempo_emprego                                        0.0595      0.002     28.433      0.000       0.055       0.064
qt_pessoas_residencia                    

In [73]:
#0.01
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , L1_wt = 1.0 
                       , alpha =  0.01)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     269.2
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          7.17e-278
Time:                        21:25:37   Log-Likelihood:                -3432.1
No. Observations:                3107   AIC:                             6878.
Df Residuals:                    3101   BIC:                             6920.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.1830      0.068    105.940      0.000       7.050       7.316
sexo[T.M]                                            0.7994      0.028     28.617      0.000       0.745       0.854
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                              0.0995      0.028      3.582      0.000       0.045       0.154
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                               0          0        nan        nan           0           0
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                           0.0491      0.018      2.743      0.006       0.014       0.084
idade                                                0.0058      0.002      3.733      0.000       0.003       0.009
tempo_emprego                                        0.0596      0.002     28.572      0.000       0.055       0.064
qt_pessoas_residencia                    

In [72]:
#0.05
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , L1_wt = 1.0
                       , alpha = 0.05)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     396.2
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          5.06e-276
Time:                        21:24:19   Log-Likelihood:                -3442.3
No. Observations:                3107   AIC:                             6895.
Df Residuals:                    3103   BIC:                             6925.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.3033      0.061    119.238      0.000       7.183       7.423
sexo[T.M]                                            0.7995      0.028     28.568      0.000       0.745       0.854
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                               0          0        nan        nan           0           0
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                                0          0        nan        nan           0           0
idade                                                0.0050      0.001      3.407      0.001       0.002       0.008
tempo_emprego                                        0.0596      0.002     28.545      0.000       0.056       0.064
qt_pessoas_residencia                    

In [45]:
#0.1
reg = md.fit_regularized(method = 'elastic_net'
                       , refit = True
                       , L1_wt = 1.0
                       , alpha = 0.1)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     398.8
Date:                Sun, 13 Mar 2022   Prob (F-statistic):          3.05e-277
Time:                        21:10:21   Log-Likelihood:                -3391.5
No. Observations:                3091   AIC:                             6793.
Df Residuals:                    3087   BIC:                             6823.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.2761      0.061    119.711      0.000       7.157       7.395
sexo[T.M]                                            0.7881      0.028     28.633      0.000       0.734       0.842
posse_de_veiculo[T.True]                                  0          0        nan        nan           0           0
posse_de_imovel[T.True]                                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                                 0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                               0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                              0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]                         0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Primário]                     0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Pós graduação]                0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior completo]            0          0        nan        nan           0           0
C(educacao, Treatment(2))[T.Superior incompleto]          0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                               0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                               0          0        nan        nan           0           0
C(estado_civil)[T.União]                                  0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                                  0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                                0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]                         0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                             0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]                       0          0        nan        nan           0           0
qtd_filhos                                                0          0        nan        nan           0           0
idade                                                0.0057      0.001      3.867      0.000       0.003       0.009
tempo_emprego                            

<p> <b>O melhor R² é com o alpha = 0, no valor de 0.358.<b></p>

### Questão 4

In [75]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            print(included+[new_column])
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [76]:
test.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [77]:
X = test[['sexo', 'posse_de_veiculo','posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao','estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego','qt_pessoas_residencia', 'renda']]
X

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
3730,F,True,True,0,Assalariado,Secundário,Casado,Casa,29,5.693151,2.0,1706.92
6573,M,False,True,0,Assalariado,Secundário,Casado,Casa,58,4.164384,2.0,15702.40
14456,F,False,True,0,Servidor público,Secundário,Casado,Casa,53,9.183562,2.0,7357.24
7636,F,False,True,1,Assalariado,Secundário,Casado,Casa,35,9.120548,3.0,2478.46
57,F,True,True,0,Assalariado,Secundário,Solteiro,Casa,52,10.720548,1.0,2959.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4895,F,True,True,0,Assalariado,Secundário,Casado,Casa,48,0.841096,2.0,7406.64
12981,M,False,True,1,Assalariado,Secundário,Separado,Casa,45,2.410959,2.0,7274.41
13770,F,False,True,0,Empresário,Superior completo,Casado,Casa,48,6.531507,2.0,10251.97
12368,M,True,True,1,Assalariado,Superior completo,Casado,Casa,45,9.454795,3.0,10615.12


In [78]:
y = test['renda']

In [79]:
result = stepwise_selection(X, y)

['idade']
['qtd_filhos']
['tipo_renda']


<ipython-input-75-306ed999d47b>:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).